In [5]:
from imp import reload

import torch
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import random
import datetime
from tqdm import tqdm
import argparse, os, math
import numpy as np
from hpcp_loader_for_softdtw import *
import models.BaseSPPNet as models
from config import DefaultConfig, opt
from utility import *
import json
import torch.nn.functional as F
from torch.nn import *
import pandas as pd
import seaborn as sns
import resource
import librosa

In [ ]:
# argparse
save_dir = None
lr = 1e-4
saved_model_path = None
epochs = 0
epochs_finished = 0
num_floor = -1
BATCH_SIZE = 16
overlap = 4
threshold = 0.05

print('--------------ArgParse--------------')

parser = argparse.ArgumentParser()

parser.add_argument('-d', '--save_dir', help='目标存储目录')
parser.add_argument('-s', '--saved_model', help='以训练过的模型')
parser.add_argument('--lr', type=float, help='学习率')
parser.add_argument('-e', '--epochs', type=int, help='有几个epoch')
# parser.add_argument('--epochs_finished', type=int, help='已经完成了几个epoch')
parser.add_argument('-f', '--out_floor', type=int, help='输出在第几层(0，1，2，3)')
parser.add_argument('-g', '--gpu', help='要用的gpu号')
parser.add_argument('-b', '--batch_size', type=int, help='batch_size')
parser.add_argument('-o', '--overlap', type=int, help='切出训练数据时，跳步占全部长度的几分之一')
parser.add_argument('-t','--threshold', type=float, help='生成结果用的阈值')
# parser.add_argument('--loss', type=int, help='损失函数')
parser.add_argument('--vt', help='验证/测试集')
parser.add_argument('--label', help='label的类型')
parser.add_argument('--fold', help='fold的类型')

args = parser.parse_args()

assert args.save_dir != None, ('请输入存储目录')
if not os.path.exists(args.save_dir):
    print(f'Save_dir {args.save_dir} does not exist. Will create one.')
    save_dir = args.save_dir
    os.mkdir(save_dir)
else:
    print(f'Save_dir {args.save_dir} already exist. Will save to this directory.')
    save_dir = args.save_dir


if args.saved_model == None:
    print('Don\'t use saved model, train from scratch')
else:
    print(f'Use saved model[{args.saved_model}], train from it')
saved_model_path = args.saved_model

if args.lr == None:
    print('Using default lr=1e-4')
else:
    lr = args.lr
    print(f'Using lr from command: {lr}')

assert args.epochs != None, ('请输入epochs数')
epochs = args.epochs

assert args.out_floor != None, ('请输入输出的层数')
assert args.out_floor in [0,1,2,3], ('输入的层数必须为0 1 2 3之一')
num_floor = args.out_floor

assert args.gpu != None, ('请输入要用的gpu号')
device_ids = list(map(lambda x: int(x), args.gpu.split(',')))
print(f'using device_ids: {device_ids}')
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
print(f'device count: {torch.cuda.device_count()}')
print(f'current device: {torch.cuda.current_device()}')

device = torch.device("cuda:0")

if args.batch_size == None:
    print(f'using default batch_size {BATCH_SIZE}')
else:
    BATCH_SIZE = args.batch_size
    print(f'using batch_size from command: {BATCH_SIZE}')
    
if args.overlap == None:
    print(f'using default overlap {overlap}')
else:
    overlap = args.overlap
    print(f'using overlap from command: {overlap}')
overlap = 258//overlap

if args.threshold == None:
    print(f'using default threshold {threshold}')
else:
    threshold = args.threshold
    print(f'using threshold from command: {threshold}')
    
# if args.loss == 0:
#     loss_fn = loss_function.CrossEntropyLoss_Origin()
# elif args.loss == 1:
#     loss_fn = loss_function.CrossEntropyLoss_for_FA_CE()
# elif args.loss == 2:
#     loss_fn = loss_function.CrossEntropyLoss_for_FA_CE_VNV()
# elif args.loss == 3:
#     loss_fn = loss_function.CrossEntropyLoss_for_FA_CE_TF()
# elif args.loss == 4:
#     loss_fn = loss_function.CrossEntropyLoss_for_FA_CESQ_TF()
# else:
#     assert False, ('损失函数代号不在范围内')
loss_fn = nn.BCELoss()
print(f'Using loss_function: {loss_fn.__class__.__name__}')

assert args.vt[0]!=args.vt[1], ('验证集、测试集不能一样')
assert args.vt[0] in '0123456789', ('验证集index out of range')
assert args.vt[1] in '0123456789', ('测试集index out of range')
valid_fold_index_list = [int(args.vt[0])]
test_fold_index_list = [int(args.vt[1])]
train_fold_index_list = [i for i in range(10) if (i not in valid_fold_index_list and i not in test_fold_index_list)]
print(f'\ttrain: {train_fold_index_list}\n\tvalid: {valid_fold_index_list}\n\ttest: {test_fold_index_list}')

assert args.label in ['origin', 'real_one_hot', '3bin', '1bin'], ('label类型不在规定范围内')
label_kind = args.label
print(f'label kind: {label_kind}')

fold_version = args.fold
print(f'fold version: {fold_version}')

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = '3,4,5'
device_ids = [0,1]

kwargs = {'model': 'NeuralDTW_CNN_Mask_dilation_SPP6', 'num_workers': 0, 'batch_size': 10, 'is_label': True,
              'is_random': False, 'notes': 'mask', 'save_model': True, 'manner': 'train',
              'params': 'params/neuraldtw/mask0', 'test_length': 400, 'zo': False }

opt.feature = 'cqt'
opt.notes = 'SoftDTW'
opt.model = 'SoftDTW'
opt.batch_size = 'batch_size'
opt._parse(kwargs) # opt.model = 'NeuralDTW_CNN_Mask_dilation_SPP6'

+------------------------------------------------------+
| user config:
| model NeuralDTW_CNN_Mask_dilation_SPP6
| feature cqt
| load_model_path None
| load_latest False
| batch_size 10
| use_gpu True
| num_workers 0
| max_epoch 20
| lr 0.001
| lr_decay 0.8
| weight_decay 1e-05
| notes mask
| debug_file params/debug_file.txt
+------------------------------------------------------+


/S3/DAA/wxy/cover_song_identification/similarity/config.py:33: UserWarning: Warning: opt has not attribut is_label
  warnings.warn("Warning: opt has not attribut %s" % k)
/S3/DAA/wxy/cover_song_identification/similarity/config.py:33: UserWarning: Warning: opt has not attribut is_random
  warnings.warn("Warning: opt has not attribut %s" % k)
/S3/DAA/wxy/cover_song_identification/similarity/config.py:33: UserWarning: Warning: opt has not attribut save_model
  warnings.warn("Warning: opt has not attribut %s" % k)
/S3/DAA/wxy/cover_song_identification/similarity/config.py:33: UserWarning: Warning: opt has not attribut manner
  warnings.warn("Warning: opt has not attribut %s" % k)
/S3/DAA/wxy/cover_song_identification/similarity/config.py:33: UserWarning: Warning: opt has not attribut params
  warnings.warn("Warning: opt has not attribut %s" % k)
/S3/DAA/wxy/cover_song_identification/similarity/config.py:33: UserWarning: Warning: opt has not attribut test_length
  warnings.warn("Warning: op

In [7]:
saved_model_path = None

model = getattr(models, opt.model)(None)
model = model.cuda()
model = nn.DataParallel(model)
# model = model.to(device)

if saved_model_path != None:
    print(f'loading model from {saved_model_path}...')
    #model = torch.load(saved_model_path)
    model.load_state_dict(torch.load(saved_model_path))
else:
    print('raw model')

loss_fn = nn.BCELoss()
loss_fn = loss_fn.cuda()
loss_fn = nn.DataParallel(loss_fn)
# loss_fn = loss_fn.to(device)

raw model


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)# , weight_decay=opt.weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=10,
                                                               verbose=True, min_lr=5e-10)# 5e-6

In [9]:
# dataloader
print(f'{datetime.datetime.now()} - Preparing train_dataloader...')
train_data = triplet_CQT(out_length=400, is_label=kwargs['is_label'], is_random=kwargs['is_random'])
train_dataloader = DataLoader(train_data, opt.batch_size, shuffle=True, num_workers=opt.num_workers)

2021-09-20 17:15:25.102465 - Preparing train_dataloader...


In [10]:
# 多个epoch的训练+测试
def train(max_epoch):
    model.train()
    model.module.model.train()
    best_MAP = 0
    for epoch in range(max_epoch):
        print(f'---------epoch {epoch}----------')
        running_loss = 0
        num = 0
        for ii, (a, p, n, la, lp, ln) in tqdm(enumerate(train_dataloader)):
            B, _, _, _ = a.shape
            target = torch.cat((torch.ones(B), torch.zeros(B))).cuda()
            # train model
            # a = a.requires_grad_().to(opt.device)
            # p = p.requires_grad_().to(opt.device)
            # n = n.requires_grad_().to(opt.device)
            a, p, n = a.cuda(device=device_ids[0]), p.cuda(device=device_ids[0]), n.cuda(device=device_ids[0])

            optimizer.zero_grad()
            pred = model(a, p, n)
            pred = pred.squeeze(1)
            loss = loss_fn(pred, target)
            print(loss)
            loss = loss.sum()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            num += B

            if (ii+1) % 4 == 0 :
                print(num)
                running_loss /= num
                print("train_loss:", running_loss)

#                     MAP = 0
#                     print("Youtube350:")
#                     MAP += val_slow_batch(model, val_dataloader, batch=100, is_dis=kwargs['zo'])
#                     print("CoverSong80:")
#                     MAP += val_slow_batch(model, val_dataloader80, batch=100, is_dis=kwargs['zo'])
                # print("Youtube350:")
                # MAP += val_quick(model, val_dataloader)
                # print("CoverSong80:")
                # MAP += val_quick(model, val_dataloader80)
                # print("Marukars:")
                # MAP += val_slow_batch(model, val_dataloader_marukars, batch=100, is_dis=kwargs['zo'])
#                     if MAP > best_MAP:
#                         best_MAP = MAP
#                         print('*****************BEST*****************')
#                     if kwargs['save_model'] == True:
#                         if parallel:
#                             model.module.save(opt.notes)
#                         else:
#                             model.save(opt.notes)
                scheduler.step(running_loss)
                running_loss = 0
                num = 0

            
#         if loss.item() != loss.item():
#             print('Train NaN!')
#             print(loss_gpus)
#             print('====================================================')
#             torch.save((X,y), os.path.join(save_dir, 'Xy_nan.pt'))
#             torch.save(model.state_dict(), os.path.join(save_dir, 'model_nan.pth'))
#             exit()

            
#     scheduler.step()
#     return loss_total/batch_num

In [11]:
def test(dataloader, model, loss_fn, out_floor):
    
    with torch.no_grad():
    
        size = len(dataloader.dataset)
        batch_num = math.ceil(size/dataloader.batch_size)

        model.eval()
        test_loss = 0
        test_loss_v, test_loss_nv = 0, 0
        test_loss_f, test_loss_t = 0, 0
        oa_avg, vr_avg, vfa_avg, rpa_avg, rca_avg = 0, 0, 0, 0, 0
    
    
        for X, y in dataloader:
            # X, y = X.to(device), y.to(device)
            X, y = X.cuda(device=device_ids[0]), y.cuda(device=device_ids[0])
            Xpred = model(X, out_floor)
            Xout = utils.salience_to_output(Xpred.clone().detach(), threshold=threshold)
            
            if out_floor == 0:
                loss = loss_fn(Xpred, y, threshold)
                oa, vr, vfa, rpa, rca = evaluate.evaluate(Xout, y, out_floor)
            else:
                y_downsample = utils.downsample(y, out_floor)
                loss = loss_fn(Xpred, y_downsample, threshold)
                oa, vr, vfa, rpa, rca = evaluate.evaluate(Xout, y_downsample, out_floor)
                
            if args.loss == 2:
                loss_gather = (loss[0].sum()+loss[2].sum())/(loss[1].sum()+loss[3].sum())
                loss_v_gather = loss[0].sum()/loss[1].sum() if loss[1].sum() else loss[1].sum()*0
                loss_nv_gather = loss[2].sum()/loss[3].sum() if loss[3].sum() else loss[3].sum()*0
                test_loss_v += loss_v_gather.item()
                test_loss_nv += loss_nv_gather.item()
            elif args.loss in [3,4]:
                loss_gather = (loss[0].sum()+loss[2].sum())/(loss[1].sum()+loss[3].sum())
                loss_f_gather = loss[0].sum()/loss[1].sum() if loss[1].sum() else loss[1].sum()*0
                loss_t_gather = loss[2].sum()/loss[3].sum() if loss[3].sum() else loss[3].sum()*0
                test_loss_f += loss_f_gather.item()
                test_loss_t += loss_t_gather.item()
            elif args.loss in [0,1]:
                loss_gather = loss.sum()
                
            if loss_gather.item() != loss_gather.item():
                print('Test NaN!')
                print(loss)
                print('====================================================')
                torch.save((X,y), os.path.join(save_dir, 'Xy_nan.pt'))
                torch.save(model.state_dict(), os.path.join(save_dir, 'model_nan.pth'))
                exit()
            
            test_loss += loss_gather.item()
            
    
    print(f"Test Error: Avg loss: {test_loss:.4f} \n")
    print(f"Test OA\t{oa_avg:.4f}\tVR\t{vr_avg:.4f}\tVFA\t{vfa_avg:.4f}\tRPA\t{rpa_avg:.4f}\tRCA\t{rca_avg:.4f}\n")
    
    if args.loss in [0,1,2]:
        return (test_loss, test_loss_v, test_loss_nv), oa_avg, vr_avg, vfa_avg, rpa_avg, rca_avg
    elif args.loss in [3,4]:
        return (test_loss, test_loss_f, test_loss_t), oa_avg, vr_avg, vfa_avg, rpa_avg, rca_avg

In [12]:
train(1)

---------epoch 0----------


0it [00:00, ?it/s]/home/yangds/anaconda3/envs/csi2/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/yangds/anaconda3/envs/csi2/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


tensor([11.8115,  6.9740,  0.5494], device='cuda:0', grad_fn=<GatherBackward>)


1it [00:10, 10.95s/it]

tensor([0.0079, 6.3098, 4.7275], device='cuda:0', grad_fn=<GatherBackward>)


2it [00:12,  5.56s/it]

tensor([0.8137, 1.7301, 8.7336], device='cuda:0', grad_fn=<GatherBackward>)


3it [00:14,  4.71s/it]


In [ ]:
for a,p,n,la,lp,ln in train_dataloader:
    break

In [ ]:
a, p, n = a.cuda(device=device_ids[0]), p.cuda(device=device_ids[0]), n.cuda(device=device_ids[0])

In [ ]:
model.eval()
model.module.model.eval()
pass

In [ ]:
with torch.no_grad():
    out = model(a,p,n).squeeze(0)
    print(out)

In [ ]:
# # 训练模型🌟
# 多个epoch训练，每个epoch后在验证集上测试

best_oa = 0

_, oa, _, _, _, _ = test(valid_dataloader, model, loss_fn, num_floor)
print(f'验证集原始OA: {oa:.4f}.')
best_oa = oa

for t in range(epochs_finished, epochs_finished+epochs):
    print(f"Epoch {t+1}\n-------------------------------{datetime.datetime.now()}")
    train_loss = train(train_dataloader, model, loss_fn, optimizer, scheduler_decay, num_floor)
    valid_loss, oa, _, _, _, _ = test(valid_dataloader, model, loss_fn, num_floor)
    test_loss, test_oa, _, _, _, _ = test(test_dataloader, model, loss_fn, num_floor)

    train_loss_list.append(train_loss)
    valid_oa_list.append(oa)
    test_oa_list.append(test_oa)
    if args.loss in [0,1]:
        valid_loss_list.append(valid_loss)
        test_loss_list.append(test_loss)
        scheduler_stop.step(valid_loss)
    elif args.loss == 2:
        valid_loss_list.append(valid_loss[0])
        valid_loss_v_list.append(valid_loss[1])
        valid_loss_nv_list.append(valid_loss[2])
        scheduler_stop.step(valid_loss[0])
        test_loss_list.append(test_loss[0])
        test_loss_v_list.append(test_loss[1])
        test_loss_nv_list.append(test_loss[2])
    elif args.loss in [3,4]:
        valid_loss_list.append(valid_loss[0])
        valid_loss_f_list.append(valid_loss[1])
        valid_loss_t_list.append(valid_loss[2])
        scheduler_stop.step(valid_loss[0])
        test_loss_list.append(test_loss[0])
        test_loss_f_list.append(test_loss[1])
        test_loss_t_list.append(test_loss[2])
    
    if optimizer.state_dict()['param_groups'][0]['lr']<=0:
        print(f'Early stop after {t+1} epochs.')
        break
    
    if args.loss in [0,1]:
        plt.plot(range(1,len(train_loss_list)+1), train_loss_list, 'y')
        plt.plot(range(1,len(valid_loss_list)+1), valid_loss_list, 'm')
        plt.plot(range(1,len(test_loss_list)+1), test_loss_list, 'r')
        plt.plot(range(1, len(valid_oa_list)+1), valid_oa_list, 'b')
        plt.plot(range(1, len(test_oa_list)+1), test_oa_list, 'b--')
        plt.legend(['train loss', 'valid loss', 'test_loss','valid acc','test acc'])
    elif args.loss == 2:
        plt.plot(range(1,len(train_loss_list)+1), train_loss_list, 'y')
        plt.plot(range(1,len(valid_loss_list)+1), valid_loss_list, 'm')
        plt.plot(range(1,len(valid_loss_list)+1), valid_loss_v_list, 'm--')
        plt.plot(range(1,len(valid_loss_list)+1), valid_loss_nv_list, 'm-.')
        plt.plot(range(1,len(test_loss_list)+1), test_loss_list, 'r')
        plt.plot(range(1,len(test_loss_list)+1), test_loss_v_list, 'r--')
        plt.plot(range(1,len(test_loss_list)+1), test_loss_nv_list, 'r-.')
        plt.plot(range(1, len(valid_oa_list)+1), valid_oa_list, 'b')
        plt.plot(range(1, len(test_oa_list)+1), test_oa_list, 'b--')
        plt.legend(['train loss', 'valid loss', 'valid loss(V)', 'valid loss(NV)', 'test loss', 'test loss(V)', 'test loss(NV)','valid acc','test acc'])
    elif args.loss in [3,4]:
        plt.plot(range(1,len(train_loss_list)+1), train_loss_list, 'y')
        plt.plot(range(1,len(valid_loss_list)+1), valid_loss_list, 'm')
        plt.plot(range(1,len(valid_loss_list)+1), valid_loss_f_list, 'm--')
        plt.plot(range(1,len(valid_loss_list)+1), valid_loss_t_list, 'm-.')
        plt.plot(range(1,len(test_loss_list)+1), test_loss_list, 'r')
        plt.plot(range(1,len(test_loss_list)+1), test_loss_f_list, 'r--')
        plt.plot(range(1,len(test_loss_list)+1), test_loss_t_list, 'r-.')
        plt.plot(range(1, len(valid_oa_list)+1), valid_oa_list, 'b')
        plt.plot(range(1, len(test_oa_list)+1), test_oa_list, 'b--')
        plt.legend(['train loss', 'valid loss', 'valid loss(F)', 'valid loss(T)', 'test loss', 'test loss(F)', 'test loss(T)','valid acc','test acc'])
    
    plt.savefig(os.path.join(save_dir, 'loss.png'))
    
    # 保存最优模型
    if oa > best_oa:
        with torch.no_grad():
            print(f'[in epoch {t+1}, OA got new best: from {best_oa:.4f} to {oa:.4f}. Will save the model]')
            best_oa = oa
            # torch.save(model, os.path.join(save_dir, f'model_floor{num_floor}_best.pth'))
            torch.save(model.state_dict(), os.path.join(save_dir, f'model_floor{num_floor}_best.pth'))
    
print("Done!")